# Задание

Необходимо объединить три машины в кластер. 
На главной машине запустить специальный сервис проверки. Этот сервис будет следить за происходящим на кластере и если заданнная в этом задании конфигурация будет выставлена корректно, то он выдаст правильный ответ.
Этот правильный ответ необходимо будет записать здесь в файл `result.txt`. 

Комада для запуска сервиса

```bash
docker service create \
  --name=supervisor \
  --publish=9090:9090/tcp \
  --constraint=node.role==manager \
  --mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  adkosmos/supervisor:latest
```

После этого можно открыть порт 9090 и посмотреть, что покажет этот сервис. Как только корректная конфигурация будет запущена, по этому порту можно будет получить правильный ответ. Если в конфигурации ошибка или чего-то не хватает, то он покажет эту информацию.


Требуемая конфигурация кластера - необходимо запустить два сервиса
* Сервисы должны называться `database` и `application`. 
* Для `database` необходимо использовать образ `mongo:4.2`, а для `application` - `adicom/admin-mongo:latest`.
* Для `application` должны быть проброшен порт `1234` -> `1234`
* Для `application` должна быть подключена конфигурация через инструмент `configs` по пути `/config/app.json`. Само содержимое этого файла конфигурации указано ниже
* `application` должен быть запущен в двух копиях - то есть параметр репликации выставлен в 2.

Файл конфигурации для `application`
```json
{
    "app": {
        "host": "database",
        "port": 27017
    }
}
```

## Решение

**docker-compose.yaml**
```yaml
version: "3.7"

services:
  database:
    image: mongo:4.2  
  application:
    image: adicom/admin-mongo:latest
    deploy:
      mode: replicated
      replicas: 2  # Добавляем 2 реплики этого сервиса
    configs:
      - source: file-server-config
        target: /config/app.json
    ports:
      - 1234:1234

configs:  # Задаем файлы конфигураций, которые необходимо распределить по кластеру
  file-server-config:
    file: ./config/app.json
```

In [1]:
%%writefile result.txt
I’m a dumb teen boy I eat sticks and rocks and mud I don’t care about the government And I really need a hug

Writing result.txt


In [2]:
!cat result.txt

I’m a dumb teen boy I eat sticks and rocks and mud I don’t care about the government And I really need a hug